In [ ]:
import numpy as np #tratamentos numéricos
import pandas as pd #dataframes
import seaborn as sns #distribuições
import matplotlib.pyplot as plt #gráficos
from sklearn.metrics import r2_score #calculo do r2
from sklearn.decomposition import PCA #pca
from statsmodels.formula.api import ols #regressão linear
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.svm import SVC #SVM
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import mean_squared_error #erro quadrático médio
from sklearn.preprocessing import StandardScaler #padronização
from sklearn.linear_model import LinearRegression #regressão linear
from sklearn.neighbors import KNeighborsClassifier #k-vizinhos
from sklearn.model_selection import cross_val_score #validação cruzada
from sklearn.model_selection import StratifiedKFold #k-vizinhos
from sklearn.model_selection import train_test_split #dividir base em treino e teste

from sklearn.metrics import plot_confusion_matrix

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, learning_curve, KFold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
import random
from sklearn.svm import SVC
import sklearn.metrics as sk

# Descrição dos Dados

In [ ]:
df = pd.read_csv('sgemm_product.csv',sep=',')
#df1 = pd.read_csv('sgemm_product.csv',sep=',')
#df = df1.sample(frac=0.4)
print('Número de linhas e colunas:',df.shape)
df.head()

##### Significado dos dados da base
    - Input:
        MWG, NWG: per-matrix 2D tiling at workgroup level: {16, 32, 64, 128} (integer)
        KWG: inner dimension of 2D tiling at workgroup level: {16, 32} (integer)
        MDIMC, NDIMC: local workgroup size: {8, 16, 32} (integer)
        MDIMA, NDIMB: local memory shape: {8, 16, 32} (integer)
        KWI: kernel loop unrolling factor: {2, 8} (integer)
        VWM, VWN: per-matrix vector widths for loading and storing: {1, 2, 4, 8} (integer)
        STRM, STRN: enable stride for accessing off-chip memory within a single thread: {0, 1} (categorical)
        SA, SB: per-matrix manual caching of the 2D workgroup tile: {0, 1} (categorical)
    
    - Output:
        Run1, Run2, Run3, Run4: performance times in milliseconds for 4 independent runs using the same parameters.
<img src='./img/img-SGEMM-GPU.gif'>

In [ ]:
#descrição estatística
df.describe()

# Tratamento e transformação dos dados

## Dados duplicados

In [ ]:
#verificando se temos duplicidade dos dados
df[df.duplicated() == True]

## Dados faltantes

In [ ]:
#verificando se temos dados faltantes
df.isnull().sum()

## Outliers

In [ ]:
#verificando outliers dos tempos de resposta
plt.figure(figsize=(16,5))
sns.boxplot(x="variable", y="value", data=pd.melt(df.iloc[:,-4:]))
plt.show()

In [ ]:
#detectando e removendo outliers
def detect_remove_outlier(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3-Q1
    mask = ((data > (Q1 - 1.5 * IQR)) & (data < (Q3 + 1.5 * IQR)))
    data_clean = data[mask]
    return data_clean

df_clean = detect_remove_outlier(df)
df_clean.head()

In [ ]:
#verificando dados faltantes obtidos pela função de outlier
df_clean.isnull().sum().sort_values(ascending=False).head(18)

In [ ]:
#dropando as linhas com NaN
df_clean = df_clean.dropna()
df_clean = df_clean.reset_index()
print('Tamanho da base sem outliers:',df_clean.shape)
df_clean.isnull().sum().sort_values(ascending=False).head(18)

In [ ]:
#verificando boxplot dos tempos de resposta após tratamento de outliers
plt.figure(figsize=(16,5))
sns.boxplot(x="variable", y="value", data=pd.melt(df_clean.iloc[:,-4:]))
plt.show()

## Dados correlacionados

In [ ]:
#matriz de correlação
plt.figure(figsize=(16,5))

plt.imshow(df_clean.corr(), cmap='Reds', interpolation='none', aspect='auto')
plt.xticks(range(len(df_clean.corr())), df_clean.corr().columns, rotation='vertical')
plt.yticks(range(len(df_clean.corr())), df_clean.corr().columns)
plt.suptitle('Matriz de Correlação', fontsize=15, fontweight='bold')
plt.grid(False)
plt.colorbar()
plt.show()

In [ ]:
#p = 0.30 # correlação mínima de 30%
#var = []
#for i in df_clean.corr().iloc[:,0:15]:
#    for j in df_clean.corr().iloc[:,0:15]:
#        if(i != j):
#            if np.abs(df_clean.corr()[i][j]) > p: # se maior do que |p|
#                var.append([i,j])
#print('Variáveis mais correlacionadas:\n', var)

In [ ]:
#criando um dataframe temporario
#df_clean_temp = pd.DataFrame(df_clean)

#dropando colunas menos correlacionaveis:
#df_clean_temp.drop(df_clean.iloc[:,2:8],axis = 1, inplace = True)
#df_clean_temp.drop(df_clean.iloc[:,10:14],axis = 1, inplace = True)
#df_clean_temp.drop(df_clean.iloc[:,-3:],axis = 1, inplace = True)

#plotando correlação entre algumas variáveis que mais interessaram 
#dado o heatmap apresentado  e as variáveis mais correlacionáveis:
#fig = sns.pairplot(df_clean_temp,diag_kind='hist')

## Dados distribuídos e balanceados

In [ ]:
#verificando distribuição dos tempos de resposta
plt.figure(figsize=(16,5))
#fig = sns.pairplot(df_clean.iloc[:,-4:])[0]
#fig = df_clean.iloc[:,-4:].hist(bins=100)
fig = sns.distplot(df_clean.iloc[:,-4:])

In [ ]:
#verificando estatísticas dos atributos alvo das 4 execuções
#df_clean = df_clean.reset_index()
df_clean.iloc[:,-4:].describe()

In [ ]:
#df_clean.drop(df_clean.iloc[:,-1:],axis = 1, inplace = True)
#df_clean

In [ ]:
#Consolidando os atributos de predição em um único apenas, considerando suas médias
run = df_clean.iloc[:,-4:].mean(axis=1)

df_clean['Run (ms)'] = pd.Series(run)

In [ ]:
data = df_clean.copy()
data

In [ ]:
plt.figure(figsize=(16,5))
fig = sns.distplot(data.iloc[:,-1:])

In [ ]:
#aplicando log na variável alvo
run_log = np.log(data['Run (ms)'])
data.insert(20,'Run Log (ms)',run_log)

plt.figure(figsize=(16,5))
fig = sns.distplot(data.iloc[:,-1:])

In [ ]:
#caso queira apagar a última coluna para dar rollback nos dados
#data.drop(data.iloc[:,-1:],axis = 1, inplace = True)
data

In [ ]:
#aplicando discretização dos dados no tempo de resposta
#threshold = [0,100,200,300,400,500,600]
threshold = [3,4,5,6]
#discretização 
#run_disc = pd.cut(data['Run (ms)'], bins=threshold)
run_disc = pd.cut(data['Run Log (ms)'], bins=threshold)

data.insert(21,'Run Discretizado (ms)',run_disc)

#realizando o mapping numbers
data['Run Discretizado Mapping (ms)'] = data['Run Discretizado (ms)'].astype('category').cat.codes

data['Run Discretizado Mapping (ms)'].unique()

In [ ]:
#incluindo na base de dados uma transformação dos valores para torná-los um problema de classificação binária

#avg = data['Run (ms)'].mean()
#data.loc[data['Run (ms)'] <= avg, 'Run Bin'] = 0
#data.loc[data['Run (ms)'] >  avg, 'Run Bin'] = 1

avg = data['Run Log (ms)'].mean()
data.loc[data['Run Log (ms)'] <= avg, 'Run Bin'] = 0
data.loc[data['Run Log (ms)'] >  avg, 'Run Bin'] = 1

run_bin = data[['Run Bin']].values.astype('int')
data.drop(data.iloc[:,-1:],axis = 1, inplace = True)

data.insert(22,'Run Binario',run_bin)

print('Se menor ou igual que ',avg,'então será 0')
print('Se maior que ',avg,'então será 1')

data

In [ ]:
#verificando balanceamento entre as variáveis categóricas (multiclasse)
fig1 = data['Run Discretizado (ms)'].value_counts().plot(kind='bar')

In [ ]:
#verificando balanceamento entre as variáveis categóricas mapping (multiclasse)
fig = data['Run Discretizado Mapping (ms)'].value_counts().plot(kind='bar')

mask0 = (data['Run Discretizado Mapping (ms)'] == -1)
#mask1 = (data['Run Discretizado Mapping (ms)'] == 1)
#mask2 = (data['Run Discretizado Mapping (ms)'] == 2)
mask3 = (data['Run Discretizado Mapping (ms)'] == 1)
mask4 = (data['Run Discretizado Mapping (ms)'] == 0)
mask5 = (data['Run Discretizado Mapping (ms)'] == 2)
#mask6 = (data['Run Discretizado Mapping (ms)'] == 6)

print('Distribuição das classes:')
print("[000,100]: %.2f" % (len(data[mask0].index.values)/len(data['Run Discretizado Mapping (ms)']) * 100),'%')
#print("[100,200]: %.2f" % (len(data[mask1].index.values)/len(data['Run Discretizado Mapping (ms)']) * 100),'%')
#print("[200,300]: %.2f" % (len(data[mask2].index.values)/len(data['Run Discretizado Mapping (ms)']) * 100),'%')
print("[300,400]: %.2f" % (len(data[mask3].index.values)/len(data['Run Discretizado Mapping (ms)']) * 100),' %')
print("[400,500]: %.2f" % (len(data[mask4].index.values)/len(data['Run Discretizado Mapping (ms)']) * 100),' %')
print("[500,600]: %.2f" % (len(data[mask5].index.values)/len(data['Run Discretizado Mapping (ms)']) * 100),' %')
#print("[500,600]: %.2f" % (len(data[mask6].index.values)/len(data['Run Discretizado Mapping (ms)']) * 100),' %')

In [ ]:
#verificando balanceamento entre as variáveis binárias
fig = data['Run Binario'].value_counts().plot(kind='bar')

mask0 = (data['Run Binario'] == 0)
mask1 = (data['Run Binario'] == 1)

print('Distribuição das classes:')
print(" <= Média: %.2f" % (len(data[mask0].index.values)/len(data['Run Binario']) * 100),'%')
print(" >  Média: %.2f" % (len(data[mask1].index.values)/len(data['Run Binario']) * 100),'%')

In [ ]:
#separando os dados em conjunto de treinamento e testes, na proporção 80-20
perc = 0.9

#salvando y como serie classificação multiclasse
x_train_m, y_train_m = data.iloc[0:int(len(data)*perc),1:-9],data['Run Discretizado (ms)'].iloc[0:int(len(data)*perc)]
x_test_m, y_test_m = data.iloc[int(len(data)*perc):int(len(data)),1:-9],data['Run Discretizado (ms)'].iloc[int(len(data)*perc):int(len(data))]

#salvando y como serie classificação multiclasse mapping number
x_train_map, y_train_map = data.iloc[0:int(len(data)*perc),1:-9],data['Run Discretizado Mapping (ms)'].iloc[0:int(len(data)*perc)]
x_test_map, y_test_map = data.iloc[int(len(data)*perc):int(len(data)),1:-9],data['Run Discretizado Mapping (ms)'].iloc[int(len(data)*perc):int(len(data))]

#salvando y como dataframe (incluir [[]]) classificação multiclasse
#x_train, y_train = data.iloc[0:int(len(data)*0.8),1:-7],data[['Run Discretizado (ms)']].iloc[0:int(len(data)*0.8)]
#x_test, y_test = data.iloc[int(len(data)*0.8):int(len(data)),1:-7],data[['Run Discretizado (ms)']].iloc[int(len(data)*0.8):int(len(data))]

#salvando y como classificação binário
x_train_b, y_train_b = data.iloc[0:int(len(data)*perc),1:-9],data['Run Binario'].iloc[0:int(len(data)*perc)]
x_test_b, y_test_b = data.iloc[int(len(data)*perc):int(len(data)),1:-9],data['Run Binario'].iloc[int(len(data)*perc):int(len(data))]


sc = StandardScaler()
x_train_m = sc.fit_transform(x_train_m)
x_test_m = sc.fit_transform(x_test_m)

x_train_map = sc.fit_transform(x_train_map)
x_test_map = sc.fit_transform(x_test_map)

x_train_b = sc.fit_transform(x_train_b)
x_test_b = sc.fit_transform(x_test_b)

print('Tamanho da amostra de treinamento:',len(x_train_b))
print('Tamanho da amostra de teste:',len(x_test_b))

In [ ]:
#x1_train, x1_test, y1_train, y1_test = train_test_split(df_features, df_target, test_size = 0.3, random_state = 0)
#treinando um classificador
clf = SVC(gamma='auto')
clf.fit(x_train_map,y_train_map)
ZY_ = clf.predict(x_test_map)


disp = plot_confusion_matrix(clf,x_test_map,y_test_map,cmap=plt.cm.Blues,normalize='true')

In [ ]:
x_train_b

In [ ]:
#Linear SVM
print('Linear Model',end='\n')
lsvclassifier = SVC(kernel='linear')
lsvclassifier.fit(x_train_b, y_train_b)

#Applying k-Fold Cross Validation
accuracies = cross_val_score(estimator = lsvclassifier, X = x_train_b, y = y_train_b, cv = 5)
mean_svm_linear=accuracies.mean()
std_svm_linear=accuracies.std()

#After using 5 fold cross validation
print('After 5 fold cross validation:')
print('Mean of Accuracies: ',mean_svm_linear*100,end='\n')
print('Standard deviation of Accuracies',std_svm_linear*100,end='\n')

#Predict SVM
y_predl = lsvclassifier.predict(x_test_b)

#Confusion Matrix
print('Test Output:')
print('Confusion Matrix:')
print(sk.confusion_matrix(y_test_b,y_predl))
print('Classification Report:')
print(sk.classification_report(y_test_b,y_predl))
print('Accuracy: ',sk.accuracy_score(y_test_b, y_predl, normalize=True, sample_weight=None))

In [ ]:
#aplicando PCA e plotando dados com conjunto original de dados de treino (multiclasse)
pca = PCA(n_components=6, random_state=1)
pca_result = pca.fit_transform(x_train_m)

#realizando o plot
fig = plt.figure(figsize=(20,6))
sns.scatterplot(x=pca_result[:,0],y=pca_result[:,1],alpha='auto', hue=y_train_m, palette='tab10')
plt.title('Scatterplot com projeção PCA do conjunto de dados multiclasse')
plt.show()

In [ ]:
#aplicando PCA e plotando dados com conjunto original de dados de treino (multiclasse mapping number)
pca = PCA(n_components=6, random_state=1)
pca_result = pca.fit_transform(x_train_map)

#realizando o plot
fig = plt.figure(figsize=(20,6))
sns.scatterplot(x=pca_result[:,0],y=pca_result[:,1],alpha='auto', hue=y_train_map, palette='tab10')
plt.title('Scatterplot com projeção PCA do conjunto de dados multiclasse mapping number')
plt.show()

In [ ]:
#aplicando PCA e plotando dados com conjunto original de dados de treino (binário)
pca = PCA(n_components=2, random_state=1)
pca_result = pca.fit_transform(x_train_b)

#realizando o plot
fig = plt.figure(figsize=(20,6))
sns.scatterplot(x=pca_result[:,0],y=pca_result[:,1],alpha='auto', hue=y_train_b, palette='prism')
plt.title('Scatterplot com projeção PCA do conjunto de dados binário')
plt.show()

In [ ]:
#01.K-vizinhos com classificação binária

#cv_knn = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
#mauc_knn = []
#macc_knn = []
#vk = []
#for k in range(1, 21):
#    vauc_knn = []
#    vacc_knn = []
#    for train_index, test_index in cv_knn.split(x_train_b, y_train_b):
#        x_train, x_test = x_train_b[train_index], x_train_b[test_index]
#        y_train, y_test = y_train_b[train_index], y_train_b[test_index]
#        model_knn = KNeighborsClassifier(n_neighbors=k, metric = 'euclidean')
#        model_knn.fit(x_train,y_train)
#        y_pred_knn = model_knn.predict(x_test) 
#        score = accuracy_score(y_pred_knn, y_test)
#        vauc_knn.append(roc_auc_score(y_test, y_pred_knn))
#        vacc_knn.append(accuracy_score(y_pred_knn, y_test))
#    macc_knn.append(np.mean(vacc_knn))
#    mauc_knn.append(np.mean(vauc_knn))
#    vk.append(k)
    
#best_k = np.argmax(mauc_knn)+1
#print('Melhor k:', best_k, ' AUC:',mauc_knn[best_k-1])
#plt.figure(figsize=(10,5))
#plt.plot(vk, mauc_knn, '-ro', label= 'AUC')
#plt.plot(vk, macc_knn, '-bo', label = 'Accuracy')
#plt.xlabel('k', fontsize = 15)
#plt.ylabel('Score', fontsize = 15)
#plt.legend()
#plt.show()

<img src='.\img\melhor-k.PNG'>

In [ ]:
#K-vizinhos, aplicando o modelo com o melhor k

model_KNN = KNeighborsClassifier(n_neighbors = 3, metric='euclidean')
model_KNN.fit(x_train_b,y_train_b)
y_pred_KNN = model_KNN.predict(x_test_b)
score_KNN = accuracy_score(y_pred_KNN, y_test_b)

print('+--------------Kvizinhos--------------+')
print('| Accuracy: ', '%.2f' % accuracy_score(y_pred_KNN, y_test_b),'                    |')
print('| F1 score: ', '%.2f' % f1_score(y_test_b, y_pred_KNN, average='weighted', labels=np.unique(y_pred_KNN)),'                    |')
print('| Precision:', '%.2f' % precision_score(y_test_b, y_pred_KNN, average='weighted', labels=np.unique(y_pred_KNN)),'                    |')
print('| Recall:   ', '%.2f' % recall_score(y_test_b, y_pred_KNN, average='weighted', labels=np.unique(y_pred_KNN)),'                    |')
print('+-------------------------------------+')

In [ ]:
#K-vizinhos, aplicando o modelo com o melhor k para multiclasse com mapping number

model_KNN = KNeighborsClassifier(n_neighbors = 3, metric='euclidean')
model_KNN.fit(x_train_map,y_train_map)
y_pred_KNN = model_KNN.predict(x_test_map)
score_KNN = accuracy_score(y_pred_KNN, y_test_map)

print('+--------------Kvizinhos--------------+')
print('| Accuracy: ', '%.2f' % accuracy_score(y_pred_KNN, y_test_map),'                    |')
print('| F1 score: ', '%.2f' % f1_score(y_test_map, y_pred_KNN, average='weighted', labels=np.unique(y_pred_KNN)),'                    |')
print('| Precision:', '%.2f' % precision_score(y_test_map, y_pred_KNN, average='weighted', labels=np.unique(y_pred_KNN)),'                    |')
print('| Recall:   ', '%.2f' % recall_score(y_test_map, y_pred_KNN, average='weighted', labels=np.unique(y_pred_KNN)),'                    |')
print('+-------------------------------------+')

In [ ]:
#aplicando SVM

#com ponderação (pegando como base a distribuição de classes encontradas anteriormente):
peso_0 = 0.01
peso_1 = 0.05
peso_2 = 0.04
peso_3 = 0.14
peso_4 = 0.16
peso_5 = 0.60
class_weight = {0: peso_0, 1: peso_1, 2: peso_2, 3: peso_3, 4: peso_4, 5: peso_5}

model_SVM = SVC(kernel='linear', C = 0.5, 
            random_state=1,
            class_weight = class_weight)

model_SVM.fit(x_train_map,y_train_map)
y_pred_SVM = model_SVM.predict(x_test_map)

score_SVM = accuracy_score(y_pred_SVM, y_test_map)

print('+-----------------SVM-----------------+')
print('| Accuracy: ', '%.2f' % accuracy_score(y_pred_SVM, y_test_map),'                    |')
print('| F1 score: ', '%.2f' % f1_score(y_test_map, y_pred_SVM, average='weighted', labels=np.unique(y_pred_SVM)),'                    |')
print('| Precision:', '%.2f' % precision_score(y_test_map, y_pred_SVM, average='weighted', labels=np.unique(y_pred_SVM)),'                    |')
print('| Recall:   ', '%.2f' % recall_score(y_test_map, y_pred_SVM, average='weighted', labels=np.unique(y_pred_SVM)),'                    |')
print('+-------------------------------------+')

In [ ]:
#aplicando SVM

#com ponderação (pegando como base a distribuição de classes encontradas anteriormente):
peso_0 = 0.4
peso_1 = 0.6
class_weight = {0: peso_0, 1: peso_1}

model_SVM = SVC(kernel='linear', C = 0.5, 
            random_state=1,
            class_weight = class_weight)

model_SVM.fit(x_train_b,y_train_b)
y_pred_SVM = model_SVM.predict(x_test_b)

score_SVM = accuracy_score(y_pred_SVM, y_test_b)

print('+-----------------SVM-----------------+')
print('| Accuracy: ', '%.2f' % accuracy_score(y_pred_SVM, y_test_b),'                    |')
print('| F1 score: ', '%.2f' % f1_score(y_test_b, y_pred_SVM, average='weighted', labels=np.unique(y_pred_SVM)),'                    |')
print('| Precision:', '%.2f' % precision_score(y_test_b, y_pred_SVM, average='weighted', labels=np.unique(y_pred_SVM)),'                    |')
print('| Recall:   ', '%.2f' % recall_score(y_test_b, y_pred_SVM, average='weighted', labels=np.unique(y_pred_SVM)),'                    |')
print('+-------------------------------------+')

In [ ]:
#Idéias de predição na massa de dados

#Balanceamento: verificar SMOTE

# - Regressão Linear Simples 
# - Regressão Logistica

#?????? objetivo 1: estimar desempenho
# - Classificadores para estimar desempenho (defino uma classificação com faixas de tempos e aplico os classificadores)
#   - Classificação Binaria e Multiclasse
#    - Defino classes com as faixas de desempenho que eu quero
#    - Faço leave-one-out pra separar a base em treino e teste
#    - Aplico os modelos de classificadores de machine learning
#      - Redes neurais perceptron
#      - SVM
#      - KNN
#      - NNGE
#      - PCA (otimizar atrinbutos)
#      - Random Forest
#      - ResNet
#      - Inception
#    - Aplico na base de teste
#    - Comparo os modelos e acurácias

#regressão linear, ML gradient descent, árvore de decisão, redes neurais perceptron e k-vizinhos.

# objetivo secundario: redução de dimensionalidade
# tecnicas de redução do numero de atributos (rodar ML com melhores atributos e melhorar a acuracia do modelo)

# objetivo secundario: otimização do algoritmo
# descobrir os melhores parametros que minimizam o tempo de execução (melhor desempenho)

In [ ]:
#regressão linear simples com scikit-learn

y_regression = df_clean['Run1 (ms)']
x_regression = df_clean.drop(['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)','Run (ms)'],axis = 1, inplace=False)
#print(x_regression)
#y_regression = data['Run (ms)']
#x_regression2 = data.drop(['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)','Run (ms)', 'Run Discretizado (ms)'],axis = 1, inplace=False)
#print(x_regression2)
y_regression_np = y_regression.to_numpy()
x_regression_np = x_regression.to_numpy()

p = 0.2
x_regression_train, x_regression_test, y_regression_train, y_regression_test = train_test_split(x_regression_np, y_regression_np, test_size = p, random_state = 42)

lm = LinearRegression()
lm.fit(x_regression_train, y_regression_train)
y_pred_lm = lm.predict(x_regression_test)

l = plt.plot(y_pred_lm, y_regression_test, 'co')
plt.setp(l, markersize=10)
plt.setp(l, markerfacecolor='C0')

plt.ylabel("y", fontsize=15)
plt.xlabel("Prediction", fontsize=15)

# mostra os valores preditos e originais
xl = np.arange(min(y_regression_test), 1.2*max(y_regression_test),(max(y_regression_test)-min(y_regression_test))/10)
yl = xl
plt.plot(xl, yl, 'r--')

#calculando o R2 score
R2 = r2_score(y_regression_test, y_pred_lm)
print('R2:', '%.2f' % R2)
#calculando o erro quadrático médio para a regressão linear múltipla
RSME = mean_squared_error(y_regression_test, y_pred_lm)
print("RSME:", '%.2f' % RSME)

plt.show(True)       